# Day 17: Conway Cubes

In [2]:
import numpy as np
import matplotlib.pyplot as plt
initial_state = open("input/input-day-17.txt", "r").read().split("\n")[:-1]
initial_state = [list(x) for x in initial_state]
initial_state = np.array(initial_state)


#initial_state = np.array([[".","#","."],[".",".","#"],["#","#","#"]])
initial_state

array([['#', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '#', '.', '.', '#', '.', '.', '#'],
       ['.', '.', '.', '.', '#', '.', '#', '.'],
       ['.', '#', '#', '.', '.', '#', '.', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '.'],
       ['#', '.', '.', '.', '#', '#', '#', '#'],
       ['#', '#', '#', '.', '#', '#', '.', '.'],
       ['.', '#', '#', '.', '#', '.', '#', '.']], dtype='<U1')

**Rules of the game**  

If a cube is ```active``` and ```exactly 2 or 3``` of its neighbors are ```also active```, the cube ```remains active```. Otherwise, the cube becomes inactive.  

If a cube is ```inactive``` but ```exactly 3``` of its neighbors are ```active```, the cube ```becomes active```. Otherwise, the cube remains inactive.


**Thoughts**  
Which inactive cubes can we ignore? There can be no inactive cubes which are two steps further out than the furthest active cube, to keep it simple.  

Lets see it as an ever size changing uniform search cube, which size (relative to origin) is always two more than the largest coordinate of any active cube. Only check active and inactive cubes within this region. There are many more ways of optimizing this but lets start with this simple one.

In [3]:
initial_state.shape

(8, 8)

In [6]:
import itertools

# Generate every relative position to each neighbor. Should be 80st in 4 dimensions 
combinations = list(itertools.product("-01", repeat=4))

# transform each item to a list and replace "-" with -1
for i, comb in enumerate(combinations):
    comb = list(comb)
    comb = [-1 if x == "-" else int(x) for x in comb]
    combinations[i] = comb

# Remove the coordinate which is our own position, hence not a neighbor
neighbor_vectors = [x for x in combinations if x != [0,0,0,0]]
neighbor_vectors = np.array(neighbor_vectors)

print("Number of neighbours: ", len(neighbor_vectors))

Number of neighbours:  80


**How do we know how much space to allocate in our pocket universe?**  

To start with we have the ```initial_state``` which is a plane in our 3D universe. During each round we have to be able to look one step further out in all dimensions in order to look at the neighbors 

In [10]:
pocket_universe_base = {}

In [15]:
# Lest just use a key map for our 3d matrix, only one index can exist at a time anyways
def posToKey(pos):
    i = pos[0]
    j = pos[1]
    k = pos[2]
    l = pos[3]
    return str(i)+","+str(j)+","+str(k)+","+str(l)

initial_universe_size = [-15, 15]
for i in range(*initial_universe_size):
    for j in range(*initial_universe_size):
        for k in range(*initial_universe_size):
            for l in range(*initial_universe_size):
                pocket_universe_base[posToKey([i,j,k,l])] = "."

# Fill one layer of our 4D matrix with the initial state
for i,x in enumerate(initial_state):
    for j,y in enumerate(x):
        pocket_universe_base[posToKey([i,j,0,0])] = y

In [16]:
search_range_x = [0, initial_state.shape[0]]
search_range_y = [0, initial_state.shape[1]]
search_range_z = [0, 1]
search_range_w = [0, 1]

pocket_universe = pocket_universe_base.copy()

number_of_cycles = 6

for cycle in range(number_of_cycles):
    print("Cycle:", cycle+1)
    # Increase the search range by one in both directions for each new cycle.
    # TODO: Find better way of optimizing this
    search_range_x[0] -= 1
    search_range_y[0] -= 1
    search_range_z[0] -= 1
    search_range_w[0] -= 1
    search_range_x[1] += 1
    search_range_y[1] += 1
    search_range_z[1] += 1
    search_range_w[1] += 1
    
    next_pocket_universe_state = pocket_universe.copy()
    
    print("Search range x:", search_range_x)
    print("Search range y:", search_range_y)
    print("Search range z:", search_range_z)
    print("Search range w:", search_range_w)
    
    # Go through all cubes in our defined search ranges
    for x in range(*search_range_x):
        for y in range(*search_range_y):
            for z in range(*search_range_z):
                for w in range(*search_range_w):
                    cube_pos = np.array([x,y,z,w])
                    cube_pos_key = posToKey(cube_pos)
                    cube_state = pocket_universe[cube_pos_key]

                    nr_active_neighbor_cubes = 0
                    # Check cube neighbors
                    for neighbor_vec in neighbor_vectors:
                        neighbor_pos = cube_pos + neighbor_vec
                        neighbor_pos_key = posToKey(neighbor_pos)

                        neighbor_cube_state = pocket_universe[neighbor_pos_key]
                        if(neighbor_cube_state == "#"):
                            nr_active_neighbor_cubes += 1

                    if cube_state == "#":
                        if nr_active_neighbor_cubes == 2 or nr_active_neighbor_cubes == 3:
                            next_pocket_universe_state[cube_pos_key] = "#"
                        else:
                            next_pocket_universe_state[cube_pos_key] = "."

                    elif cube_state == ".":
                        if nr_active_neighbor_cubes == 3:
                            next_pocket_universe_state[cube_pos_key] = "#"
                        else:
                            next_pocket_universe_state[cube_pos_key] = "."
    
    # Assign the current pocket_universe to the new one
    pocket_universe = next_pocket_universe_state.copy()
    
    # Count nr of active cubes in pocket universe
    nr_active_cubes = 0
    for x in range(*search_range_x):
        for y in range(*search_range_y):
            for z in range(*search_range_z):
                for w in range(*search_range_w):
                    cube_pos = np.array([x,y,z,w])
                    cube_pos_key = posToKey(cube_pos)
                    cube_state = pocket_universe[cube_pos_key]
                    if cube_state == "#":
                        nr_active_cubes += 1
    
    print("Nr of active cubes:", nr_active_cubes)
    print("")

Cycle: 1
Search range x: [-1, 9]
Search range y: [-1, 9]
Search range z: [-1, 2]
Search range w: [-1, 2]
Nr of active cubes: 122

Cycle: 2
Search range x: [-2, 10]
Search range y: [-2, 10]
Search range z: [-2, 3]
Search range w: [-2, 3]
Nr of active cubes: 193

Cycle: 3
Search range x: [-3, 11]
Search range y: [-3, 11]
Search range z: [-3, 4]
Search range w: [-3, 4]
Nr of active cubes: 688

Cycle: 4
Search range x: [-4, 12]
Search range y: [-4, 12]
Search range z: [-4, 5]
Search range w: [-4, 5]
Nr of active cubes: 572

Cycle: 5
Search range x: [-5, 13]
Search range y: [-5, 13]
Search range z: [-5, 6]
Search range w: [-5, 6]
Nr of active cubes: 1992

Cycle: 6
Search range x: [-6, 14]
Search range y: [-6, 14]
Search range z: [-6, 7]
Search range w: [-6, 7]
Nr of active cubes: 1908

